In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('ResultadosElectorales_2023.csv')

## Presidente afirmativos

In [ ]:
presidente_columns = ['distrito_nombre', 'seccionprovincial_nombre', 'seccion_nombre',
       'circuito_nombre', 'mesa_id', 'mesa_tipo', 'mesa_electores',
       'agrupacion_nombre', 'votos_cantidad']

In [ ]:
presidente = df[df['cargo_id']==1]

In [ ]:
presidente_afirmativos = presidente[presidente['votos_tipo']=='POSITIVO'][presidente_columns]

In [ ]:
presidente_afirmativos.head()

#### Agregarle mesas

In [ ]:
mesas_df = presidente_afirmativos[['distrito_nombre', 'seccion_nombre', 'circuito_nombre', 
                                   'mesa_id', 'votos_cantidad']].groupby([
    'distrito_nombre', 'seccion_nombre', 'circuito_nombre', 'mesa_id'
]).sum().reset_index()

In [ ]:
mesas_df.rename(columns = {'votos_cantidad': 'votos_mesa_total'}, inplace=True)

In [ ]:
mesas_df = mesas_df[mesas_df['votos_mesa_total'] > 0]

In [ ]:
presidente_afirmativos_total = pd.merge(presidente_afirmativos, mesas_df)

In [ ]:
presidente_afirmativos_total['percentage'] = presidente_afirmativos_total[
    'votos_cantidad'] / presidente_afirmativos_total['votos_mesa_total']

In [ ]:
presidente_afirmativos_total.shape

In [ ]:
presidente_afirmativos_total.to_csv('presidente_afirmativos.csv', index=False)

## Ganadores por mesa

In [ ]:
ganadores_por_mesa = presidente_afirmativos_total.sort_values('votos_cantidad', ascending=False).groupby(
    ['distrito_nombre', 'seccion_nombre', 'circuito_nombre', 'mesa_id'
]).first().reset_index()

## Votos totales por mesa

In [ ]:
# votos_afirmativos_por_mesa = presidente_afirmativos_total.pivot(columns=['agrupacion_nombre'], 
#                 index=[ 'distrito_nombre',
#                         'seccion_nombre',
#                         'circuito_nombre',
#                         'mesa_id'],
#                values='votos_cantidad').reset_index()

## Presidente invalidos

In [ ]:
presidente_invalidos = presidente[presidente['votos_tipo']!='POSITIVO']

#### Agregarle mesas

In [ ]:
mesas_invalidos = presidente_invalidos.pivot(columns=['votos_tipo'], 
                index=[ 'distrito_nombre',
                        'seccion_nombre',
                        'circuito_nombre',
                        'mesa_id'],
               values='votos_cantidad').reset_index()

In [ ]:
total_mesas = pd.merge(mesas_df, mesas_invalidos)

In [ ]:
total_mesas.shape

In [ ]:
total_mesas['total_invalidos'] = total_mesas[
    ['COMANDO', 'IMPUGNADO', 'NULO', 'RECURRIDO']].sum(axis=1)

In [ ]:
total_mesas['total_no_afirmativos'] = total_mesas[
    ['COMANDO', 'EN BLANCO', 'IMPUGNADO', 'NULO', 'RECURRIDO']].sum(axis=1)

In [ ]:
total_data_mesas = pd.merge(total_mesas, ganadores_por_mesa)

In [ ]:
total_data_mesas.rename(columns = {'agrupacion_nombre': 'agrupacion_ganadora'}, inplace=True)

In [ ]:
total_data_mesas['invalidos_percentage'] = total_data_mesas['total_invalidos'] / total_data_mesas['votos_mesa_total']

In [ ]:
total_data_mesas.to_csv('total_data_mesas.csv', index=False)

## Votos de todos los cargos

Todos los votos positivos de todos los cargos por mesa

In [ ]:
votos_totales_por_mesa = df[df['votos_tipo']=='POSITIVO'][['distrito_nombre', 'seccion_nombre', 'circuito_nombre', 
     'mesa_id', 'votos_cantidad']].groupby(['distrito_nombre', 'seccion_nombre', 'circuito_nombre', 
     'mesa_id']).sum().reset_index()

In [ ]:
votos_totales_por_mesa.rename(columns = {'votos_cantidad': 'votos_totales_mesa'}, inplace=True)

In [ ]:
partidos_a_mostrar = ['UNION POR LA PATRIA', 'JUNTOS POR EL CAMBIO', 'LA LIBERTAD AVANZA']

In [ ]:
partidos_df = df[df['agrupacion_nombre'].isin(partidos_a_mostrar)][
    ['distrito_nombre', 'seccion_nombre', 'circuito_nombre', 
     'mesa_id', 'cargo_nombre', 'agrupacion_nombre', 'votos_cantidad']
]

In [ ]:
votos_totales_por_partido_mesa = partidos_df[['distrito_nombre', 'seccion_nombre', 'circuito_nombre', 
     'mesa_id', 'agrupacion_nombre', 'votos_cantidad']].groupby(['distrito_nombre', 'seccion_nombre', 'circuito_nombre', 
     'mesa_id', 'agrupacion_nombre']).sum().reset_index()

In [ ]:
total_votos_df = pd.merge(votos_totales_por_partido_mesa, votos_totales_por_mesa)

In [ ]:
total_votos_df.to_csv('total_votos.csv', index=False)